# Answering Business Questions using SQL
For this project we'll continue working with the *Chinook database*. This is internally provided through the Dataquest universe, thus unfortunately not publically available. I'll however provide a schema over the database to make some degree of understanding possible:

<img src="schema.png">

## Loading the database

We begin by loading the database into the notebook:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

Since a schema over the database was provided we're aware of its structure. However, if that wasn't the case the following could be done to quickly gain an overview over what tables are available:

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


# Selecting New Albums to Purchase
The Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:
- `Regal` - Hip-Hop
- `Red Tone` - Punk
- `Meteor and the Girls` - Pop
- `Slim Jim Bites` - Blues

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

We'll need to write a query to find out which genres sell the most tracks in the USA, write up a summary of your findings, and make a recommendation for the three artists whose albums we should purchase for the store.

In [38]:
%%sql
WITH us_count AS 
    (
    SELECT il.* FROM invoice i
    INNER JOIN invoice_line il on i.invoice_id = il.invoice_id
    WHERE billing_country = 'USA'
    )

SELECT g.name,
COUNT(*) nr_tracks_sold,
ROUND(CAST(COUNT(*) AS float)/(SELECT COUNT(*) FROM us_count), 3) share_us_market
FROM us_count u
INNER JOIN track t on u.track_id = t.track_id
INNER JOIN genre g on t.genre_id = g.genre_id
GROUP BY 1
ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


name,nr_tracks_sold,share_us_market
Rock,561,0.534
Alternative & Punk,130,0.124
Metal,124,0.118
R&B/Soul,53,0.05
Blues,36,0.034
Alternative,35,0.033
Pop,22,0.021
Latin,22,0.021
Hip Hop/Rap,20,0.019
Jazz,14,0.013


Using the result achieved we can now answer which three of the four genres sell the most tracks in the US, and thus should be preferred. Based on our result our recommendations are to go with the following :
1. `Red Tone` - Punk
2. `Slim Jim Bites` - Blues
3. `Meteor and the Girls` - Pop

That is, Hip-Hop is the least popular genre of the four proposed and the other three should be preferred. However, the difference between Hip-Hop and Pop are very slim, so if other circumstances points in the favor of Hip-Hop (competition etc.) this artist might as well replace the Pop artist.

It is worth noting that these four genres combined only make up about 17% of total sales, so it's worth keeping an eye out for rock artists as this genre completely dominates the market, with its 53% market share.

## Answering Employee Sales Performance
Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [62]:
%%sql
SELECT e.first_name || " " || e.last_name AS employee_name, Cast(SUM(total) AS int) total_sales, birthdate, hire_date
FROM employee e
INNER JOIN customer c ON e.employee_id = c.support_rep_id
INNER JOIN invoice i ON c.customer_id = i.customer_id
WHERE title = 'Sales Support Agent'
GROUP BY e.employee_id
ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


employee_name,total_sales,birthdate,hire_date
Jane Peacock,1731,1973-08-29 00:00:00,2017-04-01 00:00:00
Margaret Park,1584,1947-09-19 00:00:00,2017-05-03 00:00:00
Steve Johnson,1393,1965-03-03 00:00:00,2017-10-17 00:00:00


We notice significant difference in total sales between the sales support agents. Explaining this variance is a complicated question, for which we don't have enough info to fully engage in. However, there is one explanatory variable that we immediately notice might contribute to this difference; `hire_date`. There seem to be correlation between how long you've been employed and how succesfully you perform sales-wise, as the figures indicate that the earlier date one was hired, the higher the sales performance of that employee. This seems like a reasonable relationship as more time spent at a job likely implies the skills for performing the job effectively are more developed.

## Analyzing Sales by Country
Our next task is to analyze the sales data for customers from each different country. We've have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, you have been directed to calculate data, for each country, on the:
- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Because there are a number of countries with only one customer, we should group these customers as "Other" in our analysis.

In [75]:
%%sql

WITH country_filtered AS
    (
    SELECT
        CASE
            WHEN (SELECT COUNT(*)
                 FROM customer
                 WHERE c.country=country) = 1
                 THEN "Other"
            ELSE c.country
        END AS country,
        c.customer_id,
        i.*
    FROM invoice i
    INNER JOIN customer c ON c.customer_id = i.customer_id
    )

    
SELECT country, nr_customers, tot_val_sales, avg_sales_per_customer, avg_sales_per_order
FROM (SELECT country,
        COUNT(DISTINCT customer_id) nr_customers,
        ROUND(SUM(total), 2) tot_val_sales,
        ROUND(SUM(total) / count(distinct customer_id), 2) avg_sales_per_customer,
        ROUND(SUM(total) / count(distinct invoice_id), 2) avg_sales_per_order,
        CASE
            WHEN country = 'Other' THEN 1
            ELSE 0
        END AS sort
    FROM country_filtered c
    GROUP BY country
    ORDER BY sort ASC, 3 DESC);

 * sqlite:///chinook.db
Done.


country,nr_customers,tot_val_sales,avg_sales_per_customer,avg_sales_per_order
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Other,15,1094.94,73.0,7.45


## Albums vs individual tracks
The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

purchase a whole album
purchase a collection of one or more individual tracks.
The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

It is very common when you are performing an analysis to have 'edge cases' which prevent you from getting a 100% accurate answer to your question. In this instance, we have two edge cases to consider:

- Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
- Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In the first case, since our analysis is concerned with maximizing revenue we can safely ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so we can ignore this case also.

In [88]:
%%sql

WITH i_first_t AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) ft_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    is_album,
    COUNT(invoice_id) nr_invoices,
    ROUND(CAST(count(invoice_id) AS FLOAT) / (SELECT COUNT(*) FROM invoice), 3) album_percent
FROM
    (
    SELECT
        ift.*,
        CASE
            WHEN
                (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ift.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ift.ft_id
                                     ) 
                ) IS NULL
            AND
                (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ift.ft_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ift.invoice_id
                 ) IS NULL
             THEN "Yes"
             ELSE "No"
         END AS is_album
     FROM i_first_t ift
    )
GROUP BY is_album;

 * sqlite:///chinook.db
Done.


is_album,nr_invoices,album_percent
No,500,0.814
Yes,114,0.186


Albums account for 18.6% of the purchases. Ceasing to buy albums from record labels might eliminate a significant share of revenue. Based on this, my recommendation is that Chinbook should not only purchase individual tracks, but also continue with the full albums.